# Data Extraction using Python libaries

## First step:

In general, we need to extract data from the Wikipedia page on Highest-Grossing Films.
In the beginning, we need to import and install all needed libaries and upgrade some features: we wll use libary BeautifulSoup.

In [ ]:
pip install requests beautifulsoup4

In [ ]:
pip install --upgrade pip

In this section, we will import all the necessary libraries for working with data.

### Libraries to be imported:
- **requests**: For making HTTP requests.
- **beautifulsoup4**: For parsing HTML and XML documents.
- **pandas**: For working with tabular data.

In [70]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### Parsing the Highest-Grossing Films Table from Wikipedia

In this section, we will parse the table of highest-grossing films from Wikipedia using the `requests` and `BeautifulSoup` libraries.

### Steps:
1. **Fetch the webpage**: We use the `requests` library to download the HTML content of the Wikipedia page.
2. **Parse the HTML**: We use `BeautifulSoup` to parse the HTML and extract the table.
3. **Locate the table**: We identify the table by its class name and store it in a variable for further processing.

In [71]:
# URL of the Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_highest-grossing_films"

# Fetch the webpage content
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find all tables on the page
tables = soup.find_all('table')
# Locate the specific table by its class name
table = soup.find('table', {'class': 'wikitable sortable plainrowheaders sticky-header col4right col5center col6center'})

# Check if the table was found
if table:
    print("Table found!")
else:
    print("Table not found.")

Table found!


### Extracting Director and Country Information from a Movie Page

This function extracts information about the director(s) and country/countries of origin from the infobox of a movie's Wikipedia page.

### Function: `get_info_using_link(file_link)`
- **Input**: A URL (`file_link`) pointing to the Wikipedia page of a movie.
- **Output**: A tuple containing:
  - `directors`: A list of directors (if found).
  - `origin_countries`: A list of countries of origin (if found).

### Steps:
1. **Fetch the webpage**: The function uses the `requests` library to download the HTML content of the movie's Wikipedia page.
2. **Parse the HTML**: The `BeautifulSoup` library is used to parse the HTML and locate the infobox table.
3. **Extract information**: The function searches for the "Directed by" and "Country"/"Countries" rows in the infobox and extracts the relevant data.
4. **Clean the data**: The function removes any unwanted characters (e.g., `[1]`, `[2]`) from the extracted data.

In [75]:
def get_info_using_link(file_link):
    response = requests.get(file_link)
    soup = BeautifulSoup(response.content, 'html.parser')
    infobox = soup.find('table', {'class': 'infobox vevent'})
    
    directors = None
    origin_countries = None
    
    if infobox:
        for row in infobox.find_all('tr'):
            header = row.find('th')
            if header and 'Directed by' in header.text:
                td = row.find('td')
                if td.find_all('li'): 
                    directors = [li.get_text(strip=True) for li in td.find_all('li')]
                    directors = [director.split('[')[0].strip() for director in directors]
                else:    
                    directors = [row.find('td').text.strip()]
            if header and 'Countries' in header.text:
                countries_td = row.find('td')
                origin_countries = [li.get_text(strip=True) for li in countries_td.find_all('li')]
                origin_countries = [country.split('[')[0].strip() for country in origin_countries]
            if header and 'Country' in header.text:
                origin_countries = [row.find('td').text.strip()]

    return directors, origin_countries

### Parsing and Processing Wikipedia Data

This code parses a Wikipedia table containing information about movies to extract information about rank, title of film, year and box office.  

### Steps:

1. **Extracting table rows** – `find_all('tr')`.
2. **Filtering rows** – The first row (`rows[1:]`) is skipped as it contains headers.
3. **Parsing data**:
   - `rank` and `peak`: extracted from the first two columns.
   - `title` and `link`: found in the header cell (`th`).
   - `box_office`: cleaned by removing `<sup>` tags.
   - `year`: the release year of the movie.
4. **Fetching additional data** – The function `get_info_using_link()` is used to retrieve directors and production countries via the movie's Wikipedia link.
5. **Building the `data` list** – The final dataset is structured for further processing.


In [81]:
rows = table.find_all('tr')
data = []

for row in rows[1:]:
    cols = row.find_all('td')
    if len(cols) > 4:
        rank = int(cols[0].text.strip())
        peak = int(cols[1].find(string=True, recursive=False).strip())
        title = row.find('th').find('a').text.strip()
        link = row.find('th').find('a')['href']
        for sup in cols[2].find_all('sup'):
            sup.decompose()
        box_office = cols[2].text.strip()
        year = cols[3].text.strip()
        directors, origin_countries = get_info_using_link("https://en.wikipedia.org/" + link)
        data.append([rank, peak, title, directors, box_office, origin_countries, year])

### Cleaning and Updating the Film Data

In this section, we perform the following operations on the `film_data` DataFrame:

1. **Create a DataFrame**: We create a DataFrame from the provided data with columns: `rank`, `peak`, `title`, `director`, `box_office`, `country`, and `release_year`.
2. **Clean the `box_office` column**: We remove the `[TFB]` suffix from the `box_office` values using regex.
3. **Update specific rows due to different issues with html code structure of Wikipedia pages about films**:
   - Set the `country` for the film at index 12 to `['United States', 'China']`.
   - Remove the first director from the `director` list for the film at index 29.
   - Set the `director` for the film at index 33 to `['Anna Boden', 'Ryan Fleck']`.

In [82]:
film_data = pd.DataFrame(data, columns=['rank', 'peak', 'title', 'director', 'box_office', 'country',
       'release_year'])
film_data['box_office'] = film_data['box_office'].replace('[TFB]', '', regex=True)
film_data['country'][12] = ['United States', 'China']
film_data['director'][29].pop(0)
film_data['director'][33] = ['Anna Boden', 'Ryan Fleck']

C:\Users\aldir\AppData\Local\Temp\ipykernel_23892\3505874508.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  film_data['country'][12] = ['United States', 'China']
C:\Users\aldir\AppData\Local\Temp\ipykernel_23892\3505874508.py:4: Setting

In [83]:
#just a cell to show first 5 rows of table 'film_data'
print(film_data.head(5)) 

   rank  peak                         title                    director  \
0     1     1                        Avatar             [James Cameron]   
1     2     1             Avengers: Endgame  [Anthony Russo, Joe Russo]   
2     3     3      Avatar: The Way of Water             [James Cameron]   
3     4     1                       Titanic             [James Cameron]   
4     5     3  Star Wars: The Force Awakens              [J. J. Abrams]   

       box_office                          country release_year  
0  $2,923,706,026  [United Kingdom, United States]         2009  
1  $2,797,501,328                  [United States]         2019  
2  $2,320,250,281                  [United States]         2022  
3  $2,257,844,554                  [United States]         1997  
4  $2,068,223,624                  [United States]         2015  


### Installing Required Libraries

In this section, we install the necessary libraries for working with PostgreSQL and data manipulation.

### Libraries to be installed:
- **psycopg2-binary**: A PostgreSQL adapter for Python.
- **pandas**: For data manipulation and analysis.
- **sqlalchemy**: For interacting with databases using Python.


In [ ]:
pip install psycopg2-binary pandas sqlalchemy

### Loading Data into PostgreSQL

In this section, we prepare the `film_data` DataFrame and upload it to a PostgreSQL database.

### Steps:
1. **Prepare the data**:
   - Convert the `director` and `country` columns from lists to strings (if they are lists).
2. **Set up the database connection**:
   - Create a connection string using the `sqlalchemy` library.
3. **Upload the data**:
   - Use the `to_sql` method to upload the DataFrame to the `films` table in the PostgreSQL database.

### Database connection details:
- **Username**: `postgres`
- **Password**: `Jloaj098890`
- **Host**: `localhost`
- **Port**: `5432`
- **Database**: `film_data`

In [87]:
from sqlalchemy import create_engine
import pandas as pd

# Convert 'director' and 'country' columns from lists to strings
film_data['director'] = film_data['director'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
film_data['country'] = film_data['country'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# Database connection details
username = 'postgres'
password = 'Jloaj098890'
host = 'localhost'
port = 5432
database = 'film_data'

# Create the connection string
connection_string = f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}'

# Create the SQLAlchemy engine
engine = create_engine(connection_string)

# Upload the DataFrame to the PostgreSQL database
film_data.to_sql('films', engine, if_exists='append', index=False)

print("Data is successfully uploaded in PostgreSQL!")

Data is successfully uploaded in PostgreSQL!


### Extracting Data from PostgreSQL and Saving to JSON

In this section, we perform the following steps to extract data from a PostgreSQL table and save it to a JSON file:

### Steps:
1. **Connect to the PostgreSQL database**: We use the `psycopg2` library to establish a connection to the `film_data` database.

2. **Fetch data from the `films` table**: We execute a SQL query (`SELECT * FROM films`) to retrieve all rows from the `films` table.

3. **Save the data to a JSON file**: We use the `json` module to serialize the data and save it to a file named `films.json`.

4. **Close the database connection**: After completing the operations, we close the cursor and the database connection to free up resources.

### JSON File Structure:
The resulting JSON file (`films.json`) will contain an array of objects, where each object represents a film with the following fields:
- `rank`
- `peak`
- `title`
- `release_year`
- `director`
- `box_office`
- `country`

In [88]:
import psycopg2
import json

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    dbname="film_data",
    user="postgres",
    password="Jloaj098890",
    host="localhost",
    port="5432"
)

# Fetch data from the 'films' table
cursor = conn.cursor()
cursor.execute("SELECT * FROM films")
rows = cursor.fetchall()

# Convert rows to a list of dictionaries
data = []
for row in rows:
    data.append({
        "rank": row[1],
        "peak": row[2],
        "title": row[3],
        "release_year": row[4],
        "director": row[5],
        "box_office": row[6],
        "country": row[7]
    })

# Save the data to a JSON file
with open("films.json", "w") as f:
    json.dump(data, f, indent=4)

# Close the connection
cursor.close()
conn.close()

print("Data has been successfully exported to 'films.json'.")

Data has been successfully exported to 'films.json'.
